In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen.electronic_structure.plotter import BSPlotter
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np
with MPRester("ktopgq8J6C3OwUSa") as m:
    bandgap = m.get_bandstructure_by_material_id("mp-23205")
    bsplot = BSPlotter(bandgap)
    data = bsplot.bs_plot_data()
    print("Bande interdite : ",data.get('band_gap'),"eV")
    print(" ")
    print("Energies des points dans chaque direction, en eV:")
    print("-------------------------------------------------")
    print("Gamma-L (Bande de valence)",((data.get('energy')[0]).get('1'))[15])
    print(" ")
    print("Gamma-L (Bande de conduction)",((data.get('energy')[0]).get('1'))[16])
    print(" ")
    print("L-B1|B (Bande de valence)",((data.get('energy')[1]).get('1'))[15])
    print(" ")
    print("L-B1|B (Bande de conduction)",((data.get('energy')[1]).get('1'))[16])
    
    print(" ")
    print("B1|B-Z (Bande de valence)",((data.get('energy')[2]).get('1'))[15])
    print(" ")
    print("B1|B-Z (Bande de conduction)",((data.get('energy')[2]).get('1'))[16])
    
    print(" ")
    print("Z-Gamma (Bande de valence)",((data.get('energy')[3]).get('1'))[15])
    print(" ")
    print("Z-Gamma (Bande de conduction)",((data.get('energy')[3]).get('1'))[16])
    
    print(" ")
    print("Gamma-X|Q (Bande de valence)",((data.get('energy')[4]).get('1'))[15])
    print(" ")
    print("Gamma-X|Q (Bande de conduction)",((data.get('energy')[4]).get('1'))[16])
    
    print(" ")
    print("X|Q-F (Bande de valence)",((data.get('energy')[5]).get('1'))[15])
    print(" ")
    print("X|Q-F (Bande de conduction)",((data.get('energy')[5]).get('1'))[16])
    
    print(" ")
    print("F-P1 (Bande de valence)",((data.get('energy')[6]).get('1'))[15])
    print(" ")
    print("F-P1 (Bande de conduction)",((data.get('energy')[6]).get('1'))[16])
    
    print(" ")
    print("P1-Z|L (Bande de valence)",((data.get('energy')[7]).get('1'))[15])
    print(" ")
    print("P1-Z|L (Bande de conduction)",((data.get('energy')[7]).get('1'))[16])
    
    print(" ")
    print("Z|L-P (Bande de valence)",((data.get('energy')[8]).get('1'))[15])
    print(" ")
    print("Z|L-P (Bande de conduction)",((data.get('energy')[8]).get('1'))[16])
    print(" ")
    
    print("Distances pour chaque direction (en Angstrom):")
    print("----------------------------------------------")
    
    print("Gamma-L",((data.get('ticks')).get('distance'))[1]-((data.get('ticks')).get('distance'))[0])
    print(" ")
    print("L-B1|B",((data.get('ticks')).get('distance'))[3]-((data.get('ticks')).get('distance'))[2])
    print(" ")
    print("B1|B-Z",((data.get('ticks')).get('distance'))[4]-((data.get('ticks')).get('distance'))[3])
    print(" ")
    print("Z-Gamma",((data.get('ticks')).get('distance'))[6]-((data.get('ticks')).get('distance'))[5])
    print(" ")
    print("Gamma-X|Q",((data.get('ticks')).get('distance'))[8]-((data.get('ticks')).get('distance'))[7])
    print(" ")
    print("X|Q-F",((data.get('ticks')).get('distance'))[9]-((data.get('ticks')).get('distance'))[8])
    print(" ")
    print("F-P1",((data.get('ticks')).get('distance'))[11]-((data.get('ticks')).get('distance'))[10])
    print(" ")
    print("P1-Z|L",((data.get('ticks')).get('distance'))[13]-((data.get('ticks')).get('distance'))[12])
    print(" ")
    print("Z|L-P",((data.get('ticks')).get('distance'))[14]-((data.get('ticks')).get('distance'))[13])
    print(" ")
    
    print("Pente pour chaque direction :")
    print("-----------------------------")
    
    print("Gamma-L (Bande de valence)",(((((data.get('energy')[0]).get('1'))[15])[18]-(((data.get('energy')[0]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[1]-((data.get('ticks')).get('distance'))[0]))
    print(" ")
    print("Gamma-L (Bande de conduction)",(((((data.get('energy')[0]).get('1'))[16])[18]-(((data.get('energy')[0]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[1]-((data.get('ticks')).get('distance'))[0]))
    
    print(" ")
    print("L-B1|B (Bande de valence)",(((((data.get('energy')[1]).get('1'))[15])[len((((data.get('energy')[1]).get('1'))[15]))-1]-(((data.get('energy')[1]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[3]-((data.get('ticks')).get('distance'))[2]))
    print(" ")
    print("L-B1|B (Bande de conduction)",(((((data.get('energy')[1]).get('1'))[16])[len((((data.get('energy')[1]).get('1'))[15]))-1]-(((data.get('energy')[1]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[3]-((data.get('ticks')).get('distance'))[2]))
    
    print(" ")
    print("B1|B-Z (Bande de valence)",(((((data.get('energy')[2]).get('1'))[15])[len((((data.get('energy')[2]).get('1'))[15]))-1]-(((data.get('energy')[2]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[4]-((data.get('ticks')).get('distance'))[3]))
    print(" ")
    print("B1|B-Z (Bande de conduction)",(((((data.get('energy')[2]).get('1'))[16])[len((((data.get('energy')[2]).get('1'))[15]))-1]-(((data.get('energy')[2]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[4]-((data.get('ticks')).get('distance'))[3]))
    
    print(" ")
    print("Z-Gamma (Bande de valence)",(((((data.get('energy')[3]).get('1'))[15])[len((((data.get('energy')[3]).get('1'))[15]))-1]-(((data.get('energy')[3]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[6]-((data.get('ticks')).get('distance'))[5]))
    print(" ")
    print("Z-Gamma (Bande de conduction)",(((((data.get('energy')[3]).get('1'))[16])[len((((data.get('energy')[3]).get('1'))[15]))-1]-(((data.get('energy')[3]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[6]-((data.get('ticks')).get('distance'))[5]))
    
    print(" ")
    print("Gamma-X|Q (Bande de valence)",(((((data.get('energy')[4]).get('1'))[15])[len((((data.get('energy')[4]).get('1'))[15]))-1]-(((data.get('energy')[4]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[8]-((data.get('ticks')).get('distance'))[7]))
    print(" ")
    print("Gamma-X|Q (Bande de conduction)",(((((data.get('energy')[4]).get('1'))[16])[len((((data.get('energy')[4]).get('1'))[15]))-1]-(((data.get('energy')[4]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[8]-((data.get('ticks')).get('distance'))[7]))
    
    print(" ")
    print("X|Q-F (Bande de valence)",(((((data.get('energy')[5]).get('1'))[15])[len((((data.get('energy')[5]).get('1'))[15]))-1]-(((data.get('energy')[5]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[9]-((data.get('ticks')).get('distance'))[8]))
    print(" ")
    print("X|Q-F (Bande de conduction)",(((((data.get('energy')[5]).get('1'))[16])[len((((data.get('energy')[5]).get('1'))[15]))-1]-(((data.get('energy')[5]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[9]-((data.get('ticks')).get('distance'))[8]))
    
    print(" ")
    print("F-P1 (Bande de valence)",(((((data.get('energy')[6]).get('1'))[15])[len((((data.get('energy')[6]).get('1'))[15]))-1]-(((data.get('energy')[6]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[11]-((data.get('ticks')).get('distance'))[10]))
    print(" ")
    print("F-P1 (Bande de conduction)",(((((data.get('energy')[6]).get('1'))[16])[len((((data.get('energy')[6]).get('1'))[15]))-1]-(((data.get('energy')[6]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[11]-((data.get('ticks')).get('distance'))[10]))
    
    print(" ")
    print("P1-Z|L (Bande de valence)",(((((data.get('energy')[7]).get('1'))[15])[len((((data.get('energy')[7]).get('1'))[15]))-1]-(((data.get('energy')[7]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[13]-((data.get('ticks')).get('distance'))[12]))
    print(" ")
    print("P1-Z|L (Bande de conduction)",(((((data.get('energy')[7]).get('1'))[16])[len((((data.get('energy')[7]).get('1'))[15]))-1]-(((data.get('energy')[7]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[13]-((data.get('ticks')).get('distance'))[12]))
    
    print(" ")
    print("Z|L-P (Bande de valence)",(((((data.get('energy')[8]).get('1'))[15])[len((((data.get('energy')[8]).get('1'))[15]))-1]-(((data.get('energy')[8]).get('1'))[15])[0]))/(((data.get('ticks')).get('distance'))[14]-((data.get('ticks')).get('distance'))[13]))
    print(" ")
    print("Z|L-P (Bande de conduction)",(((((data.get('energy')[8]).get('1'))[16])[len((((data.get('energy')[8]).get('1'))[15]))-1]-(((data.get('energy')[8]).get('1'))[16])[0]))/(((data.get('ticks')).get('distance'))[14]-((data.get('ticks')).get('distance'))[13]))
    print(" ")
    
    print("Direction dans laquelle la dernière bande de valence présente le plus et le moins de dispersion :")
    print("-------------------------------------------------------------------------------------------------")
    print("Le plus de dispersion :",max(abs(-0.6209628670379439),abs(-0.3251579114958204),0.6567904486154601,0.24183965142394512,abs(-0.6985640147122752),0.5952661583533105,abs(-0.13272319828886198),0.4290748745687808,0.33174706037568324),". Il s'agit de la direction Gamma-X|Q qui est déssinée en rouge sur la figure.")
    print("Le moins de dispersion :",min(abs(-0.6209628670379439),abs(-0.3251579114958204),0.6567904486154601,0.24183965142394512,abs(-0.6985640147122752),0.5952661583533105,abs(-0.13272319828886198),0.4290748745687808,0.33174706037568324),". Il s'agit de la direction F-P1 qui est déssinée en magenta sur la figure.")
    print(" ")
    
    print("Direction dans laquelle la première bande de conduction présente le plus et le moins de dispersion :")
    print("----------------------------------------------------------------------------------------------------")
    print("Le plus de dispersion :",max(abs(-0.7815567119615497),abs(1.518259909546511),abs(-0.097440032580549),abs(-0.1397022961017525),abs(0.1708498118869188),abs(-2.671578320835226),0.5266793582891353,1.307347094232302,abs(-1.199309274187092)),". Il s'agit de la direction X|Q-F qui est déssinée en vert sur la figure.")
    print("Le moins de dispersion :",min(abs(-0.7815567119615497),1.518259909546511,abs(-0.097440032580549),abs(-0.1397022961017525),0.1708498118869188,abs(-2.671578320835226),0.5266793582891353,1.307347094232302,abs(-1.199309274187092)),". Il s'agit de la direction B1|B-Z qui est déssinée en noir sur la figure.")
    print(" ")
    print("Supposons une dispersion parabolique :")
    print("x,y Sommet de la parabole pour la bande de valence :",((data.get('ticks')).get('distance'))[7],(((data.get('energy')[4]).get('1'))[15])[0])
    print("Point parabole valence :",((data.get('ticks')).get('distance'))[8],(((data.get('energy')[4]).get('1'))[15])[len((((data.get('energy')[4]).get('1'))[15]))-1])
    print("x,y Sommet de la parabole pour la bande de conduction :",((data.get('ticks')).get('distance'))[9],(((data.get('energy')[5]).get('1'))[16])[len((((data.get('energy')[5]).get('1'))[15]))-1])
    print("Point parabole conduction :",((data.get('ticks')).get('distance'))[8],(((data.get('energy')[5]).get('1'))[16])[0])
    print("Fonction pour la bande de valence : ","y = -0.688*(x-",((data.get('ticks')).get('distance'))[7],")^2","+",(((data.get('energy')[4]).get('1'))[15])[0])
    print("Fonction pour la bande de conduction : ","y = 5.66*(x-",((data.get('ticks')).get('distance'))[9],")^2","+",(((data.get('energy')[5]).get('1'))[16])[len((((data.get('energy')[5]).get('1'))[15]))-1])
    print(" ")
    print("Masse effective :")
    print("-----------------")
    print("      ℏ^2")
    print("m* = ------")
    print("      ∂^2E")
    print("      ----")
    print("      ∂k^2")
    def function1(x):
        return -0.688*(x- 2.7620032606423957 )*(x- 2.7620032606423957 )
    def function2(x):
        return 5.66*(x- 4.248060876279751 )*(x- 4.248060876279751 ) + 2.2645999999999997
    print("La dérivée seconde pour la bande de valence vaut :",misc.derivative(function1,x0=1,n=2),"eV/(A^2)","=",(1.6E-19/1E20)*misc.derivative(function1,x0=1,n=2),"J/(m^2)")
    print("La dérivée seconde pour la bande de conduction vaut :",misc.derivative(function2,x0=1,n=2),"eV/(A^2)","=",(1.6E-19/1E20)*misc.derivative(function2,x0=1,n=2),"J/(m^2)")
    print("Pour la bande de valence on trouve : m* = ",((1.05E-34)*(1.05E-34))/(1.6E-19/1E20)*misc.derivative(function1,x0=1,n=2),"kg","=",(((1.05E-34)*(1.05E-34))/(1.6E-19/1E20)*misc.derivative(function1,x0=1,n=2)/(9.109E-31)),"m_e")
    print("Pour la bande de conduction on trouve : m* = ",((1.05E-34)*(1.05E-34))/(1.6E-19/1E20)*misc.derivative(function2,x0=1,n=2),"kg","=",(((1.05E-34)*(1.05E-34))/(1.6E-19/1E20)*misc.derivative(function2,x0=1,n=2)/(9.109E-31)),"m_e")
    plotte = bsplot.get_plot()
    abc = plt.arrow(((data.get('ticks')).get('distance'))[7],(((data.get('energy')[4]).get('1'))[15])[0],-((data.get('ticks')).get('distance'))[7]+((data.get('ticks')).get('distance'))[8],-(((data.get('energy')[4]).get('1'))[15])[0]+(((data.get('energy')[4]).get('1'))[15])[len((((data.get('energy')[4]).get('1'))[15]))-1],head_width=0.15, head_length=0.05, linewidth=2, color='r', length_includes_head=True)
    #abcd = plt.plot([((data.get('ticks')).get('distance'))[11],((data.get('ticks')).get('distance'))[10]], [(((data.get('energy')[6]).get('1'))[15])[len((((data.get('energy')[6]).get('1'))[15]))-1], (((data.get('energy')[6]).get('1'))[15])[0]], 'm-',linewidth=2)
    abcd = plt.arrow(((data.get('ticks')).get('distance'))[10],(((data.get('energy')[6]).get('1'))[15])[0],-((data.get('ticks')).get('distance'))[10]+((data.get('ticks')).get('distance'))[11],-(((data.get('energy')[6]).get('1'))[15])[0]+(((data.get('energy')[6]).get('1'))[15])[len((((data.get('energy')[6]).get('1'))[15]))-1],head_width=0.15, head_length=0.05, linewidth=2, color='m', length_includes_head=True)
    #abcde = plt.plot([((data.get('ticks')).get('distance'))[9],((data.get('ticks')).get('distance'))[8]], [(((data.get('energy')[5]).get('1'))[16])[len((((data.get('energy')[5]).get('1'))[15]))-1], (((data.get('energy')[5]).get('1'))[16])[0]], 'g-',linewidth=2)
    abcde = plt.arrow(((data.get('ticks')).get('distance'))[8],(((data.get('energy')[5]).get('1'))[16])[0],((data.get('ticks')).get('distance'))[9]-((data.get('ticks')).get('distance'))[8],(((data.get('energy')[5]).get('1'))[16])[len((((data.get('energy')[5]).get('1'))[15]))-1]-(((data.get('energy')[5]).get('1'))[16])[0],head_width=0.1, head_length=0.05, linewidth=2, color='g', length_includes_head=True)
    #abcdf = plt.plot([((data.get('ticks')).get('distance'))[4],((data.get('ticks')).get('distance'))[3]], [(((data.get('energy')[2]).get('1'))[16])[len((((data.get('energy')[2]).get('1'))[15]))-1], (((data.get('energy')[2]).get('1'))[16])[0]], 'k-',linewidth=2)
    abcdf = plt.arrow(((data.get('ticks')).get('distance'))[3],(((data.get('energy')[2]).get('1'))[16])[0],-((data.get('ticks')).get('distance'))[3]+((data.get('ticks')).get('distance'))[4],(((data.get('energy')[2]).get('1'))[16])[len((((data.get('energy')[2]).get('1'))[15]))-1]-(((data.get('energy')[2]).get('1'))[16])[0],head_width=0.1, head_length=0.05, linewidth=2, color='k', length_includes_head=True)
    plt.show()

Bande interdite :  Indirect \Gamma-(0.500,0.000,0.444) bandgap = 3.6162000000000005 eV
 
Energies des points dans chaque direction, en eV:
-------------------------------------------------
Gamma-L (Bande de valence) [7.004900000000001, 7.088600000000001, 7.2379, 7.390500000000001, 7.475200000000001, 7.3702000000000005, 7.2073, 7.196200000000001, 7.269, 7.317500000000001, 7.341700000000001, 7.346300000000001, 7.3544, 7.4443, 7.4984, 7.513400000000001, 7.508000000000001, 7.4992, 7.4955]
 
Gamma-L (Bande de conduction) [8.1594, 8.116, 8.001100000000001, 7.8529, 7.7659, 7.8833, 8.0807, 8.2111, 8.1873, 8.227, 8.2324, 8.149000000000001, 8.0867, 8.0717, 8.126800000000001, 8.2523, 8.4242, 8.6159, 8.7554]
 
L-B1|B (Bande de valence) [7.4955, 7.497, 7.5015, 7.508900000000001, 7.519400000000001, 7.5359, 7.5685, 7.6454, 7.8018, 8.001900000000001, 8.0996]
 
L-B1|B (Bande de conduction) [8.7554, 8.7538, 8.7469, 8.728900000000001, 8.6922, 8.6292, 8.534600000000001, 8.4093, 8.2645, 8.1323, 8.0996]
 
B

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:95: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:103: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in double_scalars


<Figure size 1200x800 with 1 Axes>